# Debug SqueezeNet v1.3 (Simple Task) OpenCL implement with PyOpenCL and PyTorch
Partial code are copied heavily from https://github.com/pytorch/vision/blob/master/torchvision/models/squeezenet.py  
SqueezeNet Paper:https://arxiv.org/abs/1602.07360  
SqueezeNet 1.1 model from https://github.com/DeepScale/SqueezeNet/tree/master/SqueezeNet_v1.1   
SqueezeNet 1.1 has 2.4x less computation and slightly fewer parameters than SqueezeNet 1.0, without sacrificing accuracy.

Tests con fire1 del bloque 1

In [1]:
#some set up
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
import math
import time
from time import sleep, perf_counter as pc
from matplotlib.pyplot import imshow
%matplotlib inline

## Aquí tenemos el medidor de pytorch

- En esta aproximacion tengo en cuenta el tiempo de definición de capas y la carga de pesos porque se supone que entre fire y fire estas operaciones tiene que hacerse


In [2]:
canales_iniciales=64
canales_contraidos=16
canales_finales= np.int32(canales_iniciales+canales_iniciales)


acumulado_pytorch=0
acumulado_numpy=0
idea=True

count=100
tamanyo=55

squeeze_activation = nn.ReLU(inplace=True)

for i in range(count):

    imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo))
    
    weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
    bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)
    
    weights2a=np.random.randint(10,size=(canales_finales, canales_contraidos,1,1)).astype(np.float32)  
    bias2a=np.random.randint(10,size=(canales_finales,)).astype(np.float32)    
    
    weights2b=np.random.randint(10,size=(canales_finales, canales_contraidos,3,3)).astype(np.float32)    
    bias2b=np.random.randint(10,size=(canales_finales,)).astype(np.float32)    
    
    tic=pc()
    
    squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
    squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
    squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))    

    squeeze2a=nn.Conv2d(canales_contraidos, canales_finales, kernel_size=1, bias=False)
    squeeze2a.weight = nn.Parameter(torch.from_numpy(weights2a))
    squeeze2a.bias = nn.Parameter(torch.from_numpy(bias2a))
    
    squeeze2b=nn.Conv2d(canales_contraidos, canales_finales, kernel_size=3, bias=False, padding=1)
    squeeze2b.weight = nn.Parameter(torch.from_numpy(weights2b))
    squeeze2b.bias = nn.Parameter(torch.from_numpy(bias2b))    

    imagen1  = torch.from_numpy(imagen).float()
    
    salida1=squeeze1(imagen1)
    salida1_activation=squeeze_activation(salida1)
    
    salida2a=squeeze2a(salida1_activation)
    salida2a_activation=squeeze_activation(salida2a)
    salida2b=squeeze2b(salida1_activation)
    salida2b_activation=squeeze_activation(salida2b)    
    salida2_total=torch.cat([salida2a_activation,salida2b_activation], 1)
    salida2_total_a_numpy=salida2_total.detach().numpy()
    
    toc=pc()
    
    acumulado_pytorch=toc-tic+acumulado_pytorch
    
    
print(salida2_total_a_numpy)
print ("tiempo en segundos con pytorch= ", acumulado_pytorch/count)

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


[[[[ 83366.  75924.  89036. ...  95599.  84150.  82610.]
   [ 83672.  97772. 105048. ...  96198.  96303.  84964.]
   [106752.  99979.  88342. ...  97885.  88343.  96994.]
   ...
   [ 92932.  92356.  95920. ...  95031.  89895.  94754.]
   [ 82698.  94954.  93167. ...  80386.  84900.  85957.]
   [ 89330. 106703.  96674. ...  82985.  87512. 100036.]]

  [[ 89821.  84177.  96709. ... 105773.  90519.  90222.]
   [ 91565. 108163. 115013. ... 105223. 108018.  92988.]
   [116581. 108232.  96220. ... 106616.  96809. 104665.]
   ...
   [101185. 100630. 103733. ... 101550.  98578. 105259.]
   [ 90776. 101714. 102200. ...  88197.  92968.  93545.]
   [ 98778. 114972. 105778. ...  91075.  94854. 109946.]]

  [[ 76828.  73230.  84197. ...  91031.  77511.  78786.]
   [ 80054.  90928.  99396. ...  90175.  92628.  80434.]
   [101801.  96475.  85397. ...  92089.  84631.  92290.]
   ...
   [ 85150.  87191.  87686. ...  88792.  84294.  89808.]
   [ 78234.  89438.  87918. ...  74400.  80156.  82021.]
   [ 8

## Veamos ahora fire con opencl

In [3]:
# OpenCL setup
import pyopencl as cl
import sys
sys.path.append('../common')
import deviceinfo
from time import time

#wksp = '../device/v1.3/squeezenet'

#### Step0-A: compilation for emulation

In [4]:
%%bash
aoc -march=emulator ../device/v1.3/squeezenet/squeezenet_NDRange.cl -o ../device/v1.3/squeezenet/bin_em/squeezenet_NDRange.aocx
aoc -march=emulator ../device/v1.3/squeezenet/squeezenet_ST.cl -o ../device/v1.3/squeezenet/bin_em/squeezenet_ST.aocx

aoc: OpenCL kernel compilation completed successfully.
aoc: Linking Object files....
aoc: Compiling for Emulation ....
aoc: OpenCL kernel compilation completed successfully.
aoc: Linking Object files....
aoc: Compiling for Emulation ....


#### Step0-B: compilation for simulation

In [19]:
%%bash
aoc -march=simulator -v -ghdl ../device/v1.3/squeezenet/squeezenet_NDRange.cl -o ../device/v1.3/squeezenet/bin_sim/squeezenet_NDRange.aocx -board=a10gx
aoc -march=simulator -v -ghdl ../device/v1.3/squeezenet/squeezenet_ST.cl -o ../device/v1.3/squeezenet/bin_sim/squeezenet_ST.aocx -board=a10gx


aoc: Environment checks completed successfully.
Quartus location: /home/joerock/intelFPGA_pro/21.1/quartus/bin/quartus_sh
aoc: Cached files in /var/tmp/aocl/joerock may be used to reduce compilation time
aoc: Selected target board package /home/joerock/intelFPGA_pro/21.1/hld/board/a10_ref
aoc: Selected target board a10gx
aoc: Running OpenCL parser....
aoc: OpenCL parser completed 
aoc: Linking Object files....
aoc: Optimizing and doing static analysis of code...


Compiler Warning: device/v1.3/squeezenet/squeezenet_NDRange.cl:96: declaring global arguments 'input_im', 'filter_weight' and 'filter_bias' with no 'restrict' may lead to low performance for kernel 'conv2d1x1'


aoc: Linking with IP library ...
aoc: Checking if memory usage is larger than 100%...
aoc: Memory usage is not above 100.
aoc: First stage compilation completed successfully.
aoc: Compiling for Simulator.
Quartus location: /home/joerock/intelFPGA_pro/21.1/quartus/bin/quartus_sh
Creating simulation system...
Generating simulation system...
Compiling simulation...
aoc: Simulation generation done!
Simulator flow is successful.
To execute simulator, invoke host with 
	env CL_CONTEXT_MPSIM_DEVICE_INTELFPGA=1 <host_program>
aoc: Environment checks completed successfully.
Quartus location: /home/joerock/intelFPGA_pro/21.1/quartus/bin/quartus_sh
aoc: Cached files in /var/tmp/aocl/joerock may be used to reduce compilation time
aoc: Selected target board package /home/joerock/intelFPGA_pro/21.1/hld/board/a10_ref
aoc: Selected target board a10gx
aoc: Running OpenCL parser....
aoc: OpenCL parser completed 
aoc: Linking Object files....
aoc: Optimizing and doing static analysis of code...
aoc: Link

#### Step1: OpenCL preparation

In [5]:
platforms = cl.get_platforms()
context = cl.Context(
        dev_type=cl.device_type.ALL,
        properties=[(cl.context_properties.PLATFORM, platforms[0])])
queue = cl.CommandQueue(context)
queue1 = cl.CommandQueue(context)

context

<pyopencl.Context at 0x4f6f748 on <pyopencl.Device '12th Gen Intel(R) Core(TM) i7-12650H' on 'Intel(R) OpenCL' at 0x4f7c4b8>>

#### Step 2: creat kernels
Creat & build program

In [6]:
wksp = '../device/v1.3/squeezenet/'

file_dir = wksp + 'squeezenet_NDRange.cl'

kernelSource = open(file_dir).read()
program_NDR = cl.Program(context, kernelSource).build()

file_dir = wksp + 'squeezenet_ST.cl'

kernelSource = open(file_dir).read()
program_ST = cl.Program(context, kernelSource).build()

/home/joerock/anaconda3/envs/TFM/lib/python3.8/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Creat kernels

In [7]:
conv3x3_NDR = program_NDR.conv2d3x3
conv3x3_NDR.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

maxpool_NDR = program_NDR.maxpool2d
maxpool_NDR.set_scalar_arg_dtypes([np.int32, np.int32, None, None])

conv1x1_NDR = program_NDR.conv2d1x1
conv1x1_NDR.set_scalar_arg_dtypes([np.int32, np.int32, None, None, None, None])

avgpool_NDR = program_NDR.avgpool2d
avgpool_NDR.set_scalar_arg_dtypes([None, None])

conv3x3_ST = program_ST.conv2d3x3
conv3x3_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

maxpool_ST = program_ST.maxpool2d
maxpool_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None])

conv1x1_ST = program_ST.conv2d1x1
conv1x1_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None, None, None])

avgpool_ST = program_ST.avgpool2d
avgpool_ST.set_scalar_arg_dtypes([None, None])


#### OpenCL kernel: squeezenet_NDRange.cl
conv2d3x3: 2-D 3x3 convolution.  
conv2d1x1: 2-D 1x1 convolution. kerner size 1, stride 1  
maxpool2d: 2-D max pool. kerner size 3, stride 2  
avgpool2d: 2-D average pool. kernel size 13
```C
//maxPool2d 
//kernel_size=3 stride=2
//output one feature map per kernel
__kernel void maxpool2d(
	const int input_size,
	const int output_size,
	__global const float* restrict input_im,
    __global float* restrict output_im)
{
	int channels = get_global_id(0);//get output channel index
	
	input_im += channels * input_size * input_size;
	output_im += channels * output_size * output_size;

	//loop over output feature map
	for(int i = 0; i < output_size; i++)//row
	{
		for(int j = 0; j < output_size; j++)//col
		{
			//find the max value in 3x3 reigon 
			//to be one element in the output feature map
			float tmp = 0.0;

			#pragma unroll 1
			for(int k = 0; k < 3; k++)//row
			{
				#pragma unroll 1
				for(int l = 0; l < 3; l++)//col
				{
					float value = input_im[(i * 2 + k) * input_size  + j * 2 + l ];
					if(value > tmp)
						tmp = value;
				}
			}
			//store the result to output feature map
			output_im[i * output_size + j] = tmp; 
		}
	}
}

//3x3 convolution layer
//output one feature map per kernel
__kernel void conv2d3x3(
	const int input_channels, const int input_size,
	const int pad, const int stride,
	const int start_channel, //start_channel is for 1x1 feature map in fire layer
	const int output_size,
	__global const float* restrict input_im,
	__global const float* restrict filter_weight,
	__global const float* restrict filter_bias,
	__global float *restrict output_im
	)
{
	int filter_index = get_global_id(0); //get output channel index
	int i =  get_global_id(1);

	filter_weight += filter_index * input_channels * 9;
	float bias = filter_bias[filter_index];
	output_im += (start_channel + filter_index) * output_size * output_size;
	
	//loop over output feature map
	//for(int i = 0; i < output_size; i++)
	{
		for(int j = 0; j < output_size; j++)
		{
			//compute one element in the output feature map
			float tmp = bias;
			
			//compute dot product of 2 input_channels x 3 x 3 matrix
			for(int k = 0; k < input_channels; k++)
			{
				#pragma unroll
				for(int l = 0; l < 3; l++)
				{
					int h = i * stride + l - pad;
					for(int m = 0; m < 3; m++)
					{
						int w = j * stride + m - pad;
						if((h >= 0) && (h < input_size) && (w >= 0) && (w < input_size))
						{
							tmp += input_im[k * input_size * input_size + h * input_size + w] \
                               * filter_weight[9 * k + 3 * l + m];
						}
					}
				}
			}

			//add relu activation after conv
			output_im[i * output_size + j] = (tmp > 0.0) ? tmp : 0.0;
		}
	}
}

//1x1 convolution layer
//output one feature map per kernel
__kernel void conv2d1x1(
	const int input_channels, const int input_size,
	__global float *input_im,
	__global const float4* filter_weight,
	__global const float* filter_bias,
	__global float *restrict output_im)
{
	int filter_index = get_global_id(0); // 0 - (output_channels - 1)
	int i = get_global_id(1);

	filter_weight += filter_index * input_channels;

	float bias = filter_bias[filter_index];
	
	output_im += filter_index * input_size * input_size;//start_channel is for 1x1 feature map in fire layer

	//loop over output feature map
	//for(int i = 0; i < input_size; i++)
	{
		for(int j = 0; j < input_size; j++)
		{
			float tmp = bias;
			int loc = i * input_size + j;

			for(int k = 0; k < input_channels; k++)
			{
				//float8 weight = filter_weight[k];
				//float8 feature;
				tmp += input_im[((k << 2) + 0) * input_size * input_size + loc] * filter_weight[k].s0
				     + input_im[((k << 2) + 1) * input_size * input_size + loc] * filter_weight[k].s1
					 + input_im[((k << 2) + 2) * input_size * input_size + loc] * filter_weight[k].s2
					 + input_im[((k << 2) + 3) * input_size * input_size + loc] * filter_weight[k].s3;
			}
			//add relu after conv
			output_im[i * input_size + j] = (tmp > 0.0) ? tmp : 0.0;
		}
	}
}

//last layer use a 13 x 13 avgPool layer as classifier
//one class score per kernel
__kernel void avgpool2d(
	__global const float* restrict input_im,
	__global float* restrict output_im)
{
	int class_index = get_global_id(0);//get class score index

	input_im += 169 * class_index;
	
	float tmp = 0.0f;

	for(int i = 0; i < 169; i++)
	{
		tmp += input_im[i];
	}

	output_im[class_index] = tmp / 169.0;
}
```

#### OpenCL kernel: squeezenet_ST.cl
conv2d3x3: 2-D 3x3 convolution.  
conv2d1x1: 2-D 1x1 convolution. kerner size 1, stride 1  
maxpool2d: 2-D max pool. kerner size 3, stride 2  
avgpool2d: 2-D average pool. kernel size 13
```C
//maxPool2d 
//kernel_size=3 stride=2
//output one feature map per kernel
__kernel void maxpool2d(
	const int input_size,
	const int output_size,
    const int channel_size,
	__global const float* restrict input_im,
    __global float* restrict output_im)
{

    for(int channel_index = 0; channel_index < channel_size; channel_index++)
    {
        //loop over output feature map
        for(int i = 0; i < output_size; i++)//row
        {
            for(int j = 0; j < output_size; j++)//col
            {
                //find the max value in 3x3 reigon 
                //to be one element in the output feature map
                float tmp = 0.0;

                #pragma unroll 1
                for(int k = 0; k < 3; k++)//row
                {
                    #pragma unroll 1
                    for(int l = 0; l < 3; l++)//col
                    {
                        float value = input_im[(i * 2 + k) * input_size  + j * 2 + l ];
                        if(value > tmp)
                            tmp = value;
                    }
                }
                //store the result to output feature map
                output_im[i * output_size + j] = tmp;
            }
        }
    
        input_im += input_size * input_size;
        output_im += output_size * output_size;
    }
}

//3x3 convolution layer
//output one feature map per kernel
__kernel void conv2d3x3(
	const int input_channels, const int input_size,
	const int pad, const int stride,
	const int start_channel, //start_channel is for 1x1 feature map in fire layer
	const int output_size,
    const int filter_size,
	__global const float* restrict input_im,
	__global const float* restrict filter_weight,
	__global const float* restrict filter_bias,
	__global float *restrict output_im
	)
{
	
	//filter_weight += filter_index * input_channels * 9;
	output_im += start_channel * output_size * output_size;
	
	//loop over output feature map
	for(int filter_index = 0; filter_index < filter_size; filter_index++)
	{
        float bias = filter_bias[filter_index];

		for(int i = 0; i < output_size; i++)
		{
            for(int j = 0; j < output_size; j++)
            {
                //compute one element in the output feature map
                float tmp = bias;

                //compute dot product of 2 input_channels x 3 x 3 matrix
                for(int k = 0; k < input_channels; k++)
                {
                    #pragma unroll
                    for(int l = 0; l < 3; l++)
                    {
                        int h = i * stride + l - pad;
                        for(int m = 0; m < 3; m++)
                        {
                            int w = j * stride + m - pad;
                            if((h >= 0) && (h < input_size) && (w >= 0) && (w < input_size))
                            {
                                tmp += input_im[k * input_size * input_size + h * input_size + w] \
                                   * filter_weight[9 * k + 3 * l + m];
                            }
                        }
                    }
                }

                //add relu activation after conv
                output_im[i * output_size + j] = (tmp > 0.0) ? tmp : 0.0;                 
            }
		}
        
        filter_weight += input_channels * 9;
        output_im += output_size * output_size;
	}
}

//1x1 convolution layer as a single kernel (V5)
//output one feature map per kernel

__kernel void conv2d1x1(
	const int input_channels, 
    const int input_size,
    const int filter_size,
	__global const float* restrict input_im,
	__global const float* restrict filter_weight,
	__global const float* restrict filter_bias,
	__global float *restrict output_im)
{
	// Adding restrict keyword
    //loop over filters
	for(int f_i = 0; f_i < filter_size; f_i++)
	{
        //filter_weight += f_i * input_channels;

        float bias = filter_bias[f_i];
		
        // output_im += input_size * input_size;//start_channel is for 1x1 feature map in fire layer
	
		for(int ij = 0; ij < (input_size * input_size); ij++)
		{
			float tmp = bias;
			// int loc = i * input_size + j; // this is equal to ij

			for(int k = 0; k < input_channels; k++)
			{
				//float8 weight = filter_weight[k];
				//float8 feature;
				tmp += input_im[k * input_size * input_size + ij] * filter_weight[k + f_i * input_channels];
			}
			//add relu after conv
			output_im[ij + (input_size * input_size * f_i)] = (tmp > 0.0) ? tmp : 0.0;
            //output_im[ij] = (tmp > 0.0) ? tmp : 0.0;
		}
        //filter_weight += input_channels;	
        //output_im += input_size * input_size;//start_channel is for 1x1 feature map in fire layer
	
	}
}

//last layer use a 13 x 13 avgPool layer as classifier
//one class score per kernel
__kernel void avgpool2d(
	__global const float* restrict input_im,
	__global float* restrict output_im)
{
	// int class_index = get_global_id(0);//get class score index
    
    //Since it's the final layer, we know that there are only 1000 classes
    
	//input_im += 169 * class_index;

	for(int class_index = 0; class_index < 1000; class_index++)
    {
            
        float tmp = 0.0f;

        for(int i = 0; i < 169; i++)
        {
            tmp += input_im[class_index * 169 + i];
        }

        output_im[class_index] = tmp / 169.0;
    }
}
```

Run OpenCL implement  

In [8]:
tamanyo=56 #input_size
canales_iniciales=64 #input channels and 
canales_contraidos=16 #squeeze factor
canales_expandidos=64#expand factor
canales_finales= np.int32(canales_expandidos * 2)

squeeze_activation = nn.ReLU(inplace=True)

acumulado_pytorch = 0

imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo))

weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)
weights2a=np.random.randint(10,size=(canales_expandidos, canales_contraidos,1,1)).astype(np.float32)  
bias2a=np.random.randint(10,size=(canales_expandidos,)).astype(np.float32)    
weights2b=np.random.randint(10,size=(canales_expandidos, canales_contraidos,3,3)).astype(np.float32)    
bias2b=np.random.randint(10,size=(canales_expandidos,)).astype(np.float32)

tic=pc()

squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

squeeze2a=nn.Conv2d(canales_contraidos, canales_expandidos, kernel_size=1, bias=False)
squeeze2a.weight = nn.Parameter(torch.from_numpy(weights2a))
squeeze2a.bias = nn.Parameter(torch.from_numpy(bias2a))

squeeze2b=nn.Conv2d(canales_contraidos, canales_expandidos, kernel_size=3, bias=False, padding=1)
squeeze2b.weight = nn.Parameter(torch.from_numpy(weights2b))
squeeze2b.bias = nn.Parameter(torch.from_numpy(bias2b))


imagen1  = torch.from_numpy(imagen).float()

salida1=squeeze1(imagen1)
salida1_activation=squeeze_activation(salida1)

salida2a=squeeze2a(salida1_activation)
salida2a_activation=squeeze_activation(salida2a)

salida2b=squeeze2b(salida1_activation)
salida2b_activation=squeeze_activation(salida2b)

salida2_total=torch.cat([salida2a_activation,salida2b_activation], 1)
salida2_total_a_numpy=salida2_total.detach().numpy()

toc=pc()
acumulado_pytorch=toc-tic+acumulado_pytorch

####### OPENCL COMPARISON #######


In [9]:
# NDRANGE

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1
fire1_expand1x1_weight = weights2a.reshape(-1)
fire1_expand1x1_bias = bias2a
fire1_expand3x3_weight =weights2b.reshape(-1)
fire1_expand3x3_bias = bias2b

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
h_result_fire1_expand = np.empty(1 * canales_finales * tamanyo * tamanyo).astype(np.float32)

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)
d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_weight)
d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_bias)
d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

tic2 = pc()

conv1x1_NDR(queue,(canales_contraidos, tamanyo), None, np.int32(canales_iniciales/4), tamanyo, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)

queue.finish()

conv1x1_NDR(queue1,(canales_expandidos, tamanyo), None, np.int32(canales_contraidos/4), tamanyo, d_result_fire1_squeeze, d_fire1_expand1x1_weight, d_fire1_expand1x1_bias, d_result_fire1_expand)
conv3x3_NDR(queue,(canales_expandidos, tamanyo), None, canales_contraidos, tamanyo, 1, 1, canales_expandidos, tamanyo, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()
queue1.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime = pc() - tic2


/tmp/ipykernel_7051/1897510988.py:13: RuntimeWarning: overflow encountered in cast
  h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)


In [10]:
# Simple task

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1
fire1_expand1x1_weight = weights2a.reshape(-1)
fire1_expand1x1_bias = bias2a
fire1_expand3x3_weight =weights2b.reshape(-1)
fire1_expand3x3_bias = bias2b

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
h_result_fire1_expand = np.empty(1 * canales_finales * tamanyo * tamanyo).astype(np.float32)

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)
d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_weight)
d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_bias)
d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

tic3 = pc()

conv1x1_ST(queue,(1,), None, canales_iniciales, tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)

queue.finish()

conv1x1_ST(queue,(1,), None, canales_contraidos, tamanyo, canales_expandidos, d_result_fire1_squeeze, d_fire1_expand1x1_weight, d_fire1_expand1x1_bias, d_result_fire1_expand)
conv3x3_ST(queue1,(1,), None, canales_contraidos, tamanyo, 1, 1, canales_expandidos, tamanyo, canales_expandidos, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()
queue1.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos1 = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime1 = pc() - tic3


/tmp/ipykernel_7051/1029732790.py:13: RuntimeWarning: overflow encountered in cast
  h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)


In [11]:
print ("tiempo en segundos con pytorch= ", toc-tic)
print ("tiempo en segundos con opencl (NDRANGE)=",rtime)
print ("tiempo en segundos con opencl (Simple Task)=",rtime1)

comparativa1=np.allclose(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo), veamos,rtol=1e-01, atol=1e-01)
comparativa2=np.allclose(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo), veamos1,rtol=1e-01, atol=1e-01)
comparativa3=np.allclose(veamos, veamos1,rtol=1e-01, atol=1e-01)

print("comparativa (pytorch == NDRange): ",comparativa1)
print("comparativa (pytorch == Simple Task): ",comparativa2)
print("comparativa (NDRange == Simple Task): ",comparativa3)

tiempo en segundos con pytorch=  0.0043442879996291595
tiempo en segundos con opencl (NDRANGE)= 0.013322005999725661
tiempo en segundos con opencl (Simple Task)= 0.023072717000104603
comparativa (pytorch == NDRange):  True
comparativa (pytorch == Simple Task):  True
comparativa (NDRange == Simple Task):  True


In [28]:
for i in range(canales_finales):
    for j in range(tamanyo):
        for k in range(tamanyo):
            if (abs(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k] - veamos1[i][j][k])) > 1e-01:
                print("i:", i, "j:", j, "k:", k, salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k], veamos1[i][j][k])

#### Step 3: emulation
Creat & build program

In [12]:
platforms = cl.get_platforms()
context = cl.Context(
        dev_type=cl.device_type.ALL,
        properties=[(cl.context_properties.PLATFORM, platforms[1])])
device = platforms[1].get_devices()

queue = cl.CommandQueue(context)
queue1 = cl.CommandQueue(context)

context

<pyopencl.Context at 0x6eebff8 on <pyopencl.Device 'Intel(R) FPGA Emulation Device' on 'Intel(R) FPGA Emulation Platform for OpenCL(TM)' at 0x4f871f8>>

In [13]:
wksp = '../device/v1.3/squeezenet/bin_em/'

file_dir = wksp + 'squeezenet_NDRange.aocx'

kernelSource = open(file_dir, mode='rb').read()
program_NDR = cl.Program(context, device, [kernelSource]).build()

file_dir = wksp + 'squeezenet_ST.aocx'

kernelSource = open(file_dir, mode='rb').read()
program_ST = cl.Program(context, device, [kernelSource]).build()

In [14]:
conv3x3_NDR = program_NDR.conv2d3x3
conv3x3_NDR.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

maxpool_NDR = program_NDR.maxpool2d
maxpool_NDR.set_scalar_arg_dtypes([np.int32, np.int32, None, None])

conv1x1_NDR = program_NDR.conv2d1x1
conv1x1_NDR.set_scalar_arg_dtypes([np.int32, np.int32, None, None, None, None])

avgpool_NDR = program_NDR.avgpool2d
avgpool_NDR.set_scalar_arg_dtypes([None, None])

conv3x3_ST = program_ST.conv2d3x3
conv3x3_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

maxpool_ST = program_ST.maxpool2d
maxpool_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None])

conv1x1_ST = program_ST.conv2d1x1
conv1x1_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None, None, None])

avgpool_ST = program_ST.avgpool2d
avgpool_ST.set_scalar_arg_dtypes([None, None])


Run OpenCL implement  

In [15]:
tamanyo=56 #input_size
canales_iniciales=64 #input channels and 
canales_contraidos=16 #squeeze factor
canales_expandidos=64#expand factor
canales_finales= np.int32(canales_expandidos * 2)

squeeze_activation = nn.ReLU(inplace=True)

acumulado_pytorch = 0

imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo))

weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)
weights2a=np.random.randint(10,size=(canales_expandidos, canales_contraidos,1,1)).astype(np.float32)  
bias2a=np.random.randint(10,size=(canales_expandidos,)).astype(np.float32)    
weights2b=np.random.randint(10,size=(canales_expandidos, canales_contraidos,3,3)).astype(np.float32)    
bias2b=np.random.randint(10,size=(canales_expandidos,)).astype(np.float32)

tic=pc()

squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

squeeze2a=nn.Conv2d(canales_contraidos, canales_expandidos, kernel_size=1, bias=False)
squeeze2a.weight = nn.Parameter(torch.from_numpy(weights2a))
squeeze2a.bias = nn.Parameter(torch.from_numpy(bias2a))

squeeze2b=nn.Conv2d(canales_contraidos, canales_expandidos, kernel_size=3, bias=False, padding=1)
squeeze2b.weight = nn.Parameter(torch.from_numpy(weights2b))
squeeze2b.bias = nn.Parameter(torch.from_numpy(bias2b))


imagen1  = torch.from_numpy(imagen).float()

salida1=squeeze1(imagen1)
salida1_activation=squeeze_activation(salida1)

salida2a=squeeze2a(salida1_activation)
salida2a_activation=squeeze_activation(salida2a)

salida2b=squeeze2b(salida1_activation)
salida2b_activation=squeeze_activation(salida2b)

salida2_total=torch.cat([salida2a_activation,salida2b_activation], 1)
salida2_total_a_numpy=salida2_total.detach().numpy()

toc=pc()
acumulado_pytorch=toc-tic+acumulado_pytorch

####### OPENCL COMPARISON #######


In [16]:
# NDRANGE

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1
fire1_expand1x1_weight = weights2a.reshape(-1)
fire1_expand1x1_bias = bias2a
fire1_expand3x3_weight =weights2b.reshape(-1)
fire1_expand3x3_bias = bias2b

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
h_result_fire1_expand = np.empty(1 * canales_finales * tamanyo * tamanyo).astype(np.float32)

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)
d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_weight)
d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_bias)
d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

tic2 = pc()

conv1x1_NDR(queue,(canales_contraidos, tamanyo), None, np.int32(canales_iniciales/4), tamanyo, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)

queue.finish()

conv1x1_NDR(queue1,(canales_expandidos, tamanyo), None, np.int32(canales_contraidos/4), tamanyo, d_result_fire1_squeeze, d_fire1_expand1x1_weight, d_fire1_expand1x1_bias, d_result_fire1_expand)
conv3x3_NDR(queue,(canales_expandidos, tamanyo), None, canales_contraidos, tamanyo, 1, 1, canales_expandidos, tamanyo, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()
queue1.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime = pc() - tic2


In [17]:
# Simple task

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1
fire1_expand1x1_weight = weights2a.reshape(-1)
fire1_expand1x1_bias = bias2a
fire1_expand3x3_weight =weights2b.reshape(-1)
fire1_expand3x3_bias = bias2b

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
h_result_fire1_expand = np.empty(1 * canales_finales * tamanyo * tamanyo).astype(np.float32)

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)
d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_weight)
d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_bias)
d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

tic3 = pc()

conv1x1_ST(queue,(1,), None, canales_iniciales, tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)

queue.finish()

conv1x1_ST(queue,(1,), None, canales_contraidos, tamanyo, canales_expandidos, d_result_fire1_squeeze, d_fire1_expand1x1_weight, d_fire1_expand1x1_bias, d_result_fire1_expand)
conv3x3_ST(queue1,(1,), None, canales_contraidos, tamanyo, 1, 1, canales_expandidos, tamanyo, canales_expandidos, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()
queue1.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos1 = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime1 = pc() - tic3


In [18]:
print ("tiempo en segundos con pytorch= ", toc-tic)
print ("tiempo en segundos con opencl (NDRANGE)=",rtime)
print ("tiempo en segundos con opencl (Simple Task)=",rtime1)

comparativa1=np.allclose(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo), veamos,rtol=1e-01, atol=1e-01)
comparativa2=np.allclose(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo), veamos1,rtol=1e-01, atol=1e-01)
comparativa3=np.allclose(veamos, veamos1,rtol=1e-01, atol=1e-01)

print("comparativa (pytorch == NDRange): ",comparativa1)
print("comparativa (pytorch == Simple Task): ",comparativa2)
print("comparativa (NDRange == Simple Task): ",comparativa3)

tiempo en segundos con pytorch=  0.004821141999855172
tiempo en segundos con opencl (NDRANGE)= 0.027683909000188578
tiempo en segundos con opencl (Simple Task)= 0.031077208000169776
comparativa (pytorch == NDRange):  True
comparativa (pytorch == Simple Task):  True
comparativa (NDRange == Simple Task):  True


In [28]:
for i in range(canales_finales):
    for j in range(tamanyo):
        for k in range(tamanyo):
            if (abs(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k] - veamos1[i][j][k])) > 1e-01:
                print("i:", i, "j:", j, "k:", k, salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k], veamos1[i][j][k])

#### Step 4: simulación
Creat & build program

In [20]:
platforms = cl.get_platforms()
context = cl.Context(
        dev_type=cl.device_type.ALL,
        properties=[(cl.context_properties.PLATFORM, platforms[2])])
device = platforms[2].get_devices()

queue = cl.CommandQueue(context)
queue1 = cl.CommandQueue(context)

context

<pyopencl.Context at 0x6ef5f60 on <pyopencl.Device 'SimulatorDevice : Multi-process Simulator (aclmsim0)' on 'Intel(R) FPGA SDK for OpenCL(TM)' at 0x7fd26047a0d8>>

In [21]:
wksp = '../device/v1.3/squeezenet/bin_sim/'

file_dir = wksp + 'squeezenet_NDRange.aocx'

kernelSource = open(file_dir, mode='rb').read()
program_NDR = cl.Program(context, device, [kernelSource]).build()

file_dir = wksp + 'squeezenet_ST.aocx'

kernelSource = open(file_dir, mode='rb').read()
program_ST = cl.Program(context, device, [kernelSource]).build()

/home/joerock/anaconda3/envs/TFM/lib/python3.8/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


In [22]:
conv3x3_NDR = program_NDR.conv2d3x3
conv3x3_NDR.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

maxpool_NDR = program_NDR.maxpool2d
maxpool_NDR.set_scalar_arg_dtypes([np.int32, np.int32, None, None])

conv1x1_NDR = program_NDR.conv2d1x1
conv1x1_NDR.set_scalar_arg_dtypes([np.int32, np.int32, None, None, None, None])

avgpool_NDR = program_NDR.avgpool2d
avgpool_NDR.set_scalar_arg_dtypes([None, None])

conv3x3_ST = program_ST.conv2d3x3
conv3x3_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

maxpool_ST = program_ST.maxpool2d
maxpool_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None])

conv1x1_ST = program_ST.conv2d1x1
conv1x1_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None, None, None])

avgpool_ST = program_ST.avgpool2d
avgpool_ST.set_scalar_arg_dtypes([None, None])


Run OpenCL implement  

In [23]:
tamanyo=5 #input_size
canales_iniciales=8 #input channels and 
canales_contraidos=4 #squeeze factor
canales_expandidos=4#expand factor
canales_finales= np.int32(canales_expandidos * 2)

squeeze_activation = nn.ReLU(inplace=True)

acumulado_pytorch = 0

imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo))

weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)
weights2a=np.random.randint(10,size=(canales_expandidos, canales_contraidos,1,1)).astype(np.float32)  
bias2a=np.random.randint(10,size=(canales_expandidos,)).astype(np.float32)    
weights2b=np.random.randint(10,size=(canales_expandidos, canales_contraidos,3,3)).astype(np.float32)    
bias2b=np.random.randint(10,size=(canales_expandidos,)).astype(np.float32)

tic=pc()

squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

squeeze2a=nn.Conv2d(canales_contraidos, canales_expandidos, kernel_size=1, bias=False)
squeeze2a.weight = nn.Parameter(torch.from_numpy(weights2a))
squeeze2a.bias = nn.Parameter(torch.from_numpy(bias2a))

squeeze2b=nn.Conv2d(canales_contraidos, canales_expandidos, kernel_size=3, bias=False, padding=1)
squeeze2b.weight = nn.Parameter(torch.from_numpy(weights2b))
squeeze2b.bias = nn.Parameter(torch.from_numpy(bias2b))


imagen1  = torch.from_numpy(imagen).float()

salida1=squeeze1(imagen1)
salida1_activation=squeeze_activation(salida1)

salida2a=squeeze2a(salida1_activation)
salida2a_activation=squeeze_activation(salida2a)

salida2b=squeeze2b(salida1_activation)
salida2b_activation=squeeze_activation(salida2b)

salida2_total=torch.cat([salida2a_activation,salida2b_activation], 1)
salida2_total_a_numpy=salida2_total.detach().numpy()

toc=pc()
acumulado_pytorch=toc-tic+acumulado_pytorch

####### OPENCL COMPARISON #######


In [24]:
# NDRANGE

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1
fire1_expand1x1_weight = weights2a.reshape(-1)
fire1_expand1x1_bias = bias2a
fire1_expand3x3_weight =weights2b.reshape(-1)
fire1_expand3x3_bias = bias2b

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
h_result_fire1_expand = np.empty(1 * canales_finales * tamanyo * tamanyo).astype(np.float32)

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)
d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_weight)
d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_bias)
d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

tic2 = pc()

conv1x1_NDR(queue,(canales_contraidos, tamanyo), None, np.int32(canales_iniciales/4), tamanyo, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)

queue.finish()

conv1x1_NDR(queue1,(canales_expandidos, tamanyo), None, np.int32(canales_contraidos/4), tamanyo, d_result_fire1_squeeze, d_fire1_expand1x1_weight, d_fire1_expand1x1_bias, d_result_fire1_expand)
conv3x3_NDR(queue,(canales_expandidos, tamanyo), None, canales_contraidos, tamanyo, 1, 1, canales_expandidos, tamanyo, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()
queue1.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime = pc() - tic2


/tmp/ipykernel_7051/1897510988.py:14: RuntimeWarning: overflow encountered in cast
  h_result_fire1_expand = np.empty(1 * canales_finales * tamanyo * tamanyo).astype(np.float32)


In [25]:
# Simple task

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1
fire1_expand1x1_weight = weights2a.reshape(-1)
fire1_expand1x1_bias = bias2a
fire1_expand3x3_weight =weights2b.reshape(-1)
fire1_expand3x3_bias = bias2b

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
h_result_fire1_expand = np.empty(1 * canales_finales * tamanyo * tamanyo).astype(np.float32)

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)
d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_weight)
d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_bias)
d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

tic3 = pc()

conv1x1_ST(queue,(1,), None, canales_iniciales, tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)

queue.finish()

conv1x1_ST(queue,(1,), None, canales_contraidos, tamanyo, canales_expandidos, d_result_fire1_squeeze, d_fire1_expand1x1_weight, d_fire1_expand1x1_bias, d_result_fire1_expand)
conv3x3_ST(queue1,(1,), None, canales_contraidos, tamanyo, 1, 1, canales_expandidos, tamanyo, canales_expandidos, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()
queue1.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos1 = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime1 = pc() - tic3


/tmp/ipykernel_7051/1029732790.py:14: RuntimeWarning: overflow encountered in cast
  h_result_fire1_expand = np.empty(1 * canales_finales * tamanyo * tamanyo).astype(np.float32)


In [26]:
print ("tiempo en segundos con pytorch= ", toc-tic)
print ("tiempo en segundos con opencl (NDRANGE)=",rtime)
print ("tiempo en segundos con opencl (Simple Task)=",rtime1)

comparativa1=np.allclose(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo), veamos,rtol=1e-01, atol=1e-01)
comparativa2=np.allclose(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo), veamos1,rtol=1e-01, atol=1e-01)
comparativa3=np.allclose(veamos, veamos1,rtol=1e-01, atol=1e-01)

print("comparativa (pytorch == NDRange): ",comparativa1)
print("comparativa (pytorch == Simple Task): ",comparativa2)
print("comparativa (NDRange == Simple Task): ",comparativa3)

tiempo en segundos con pytorch=  0.20604586599984032
tiempo en segundos con opencl (NDRANGE)= 124.8048321590004
tiempo en segundos con opencl (Simple Task)= 470.6451553429997
comparativa (pytorch == NDRange):  True
comparativa (pytorch == Simple Task):  True
comparativa (NDRange == Simple Task):  True


In [15]:
print("pytorch", veamos.shape, salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[0][0])
print("NDRange", veamos[0][0])
print("Simple task", veamos1[0][0])
print(imagen.shape, np.allclose(imagen, np.ones((1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32), rtol=1e-01, atol=1e-01))
#print(weights1[6])
# print(fire1_squeeze_weight)
#print(bias1[6])
# print(imagen1_obtenida)
# print(veamos2)
print(np.allclose(salida1_a_numpy, veamos1,rtol=1e-01, atol=1e-01))

pytorch (2, 5, 5) [123. 151. 184. 152. 138.]
NDRange [123. 151. 184. 152. 138.]
Simple task [ 77.  71. 126. 184. 132.]
(1, 8, 5, 5) False
False


In [28]:
for i in range(canales_finales):
    for j in range(tamanyo):
        for k in range(tamanyo):
            if (abs(salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k] - veamos1[i][j][k])) > 1e-01:
                print("i:", i, "j:", j, "k:", k, salida2_total_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k], veamos1[i][j][k])